## Semana 02 - Big Data

## Bibliotecas

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

## Setup

In [43]:
import findspark
findspark.init()

In [44]:
spark = SparkSession.builder.getOrCreate()

In [45]:
df = spark.createDataFrame([('Pedro', '4'), ('João', '5')], schema = 'nome STRING, id STRING')

In [46]:
df.show()

+-----+---+
| nome| id|
+-----+---+
|Pedro|  4|
| João|  5|
+-----+---+



## Acessando os Tipos de Dados

In [47]:
from pyspark.sql.types import *



In [48]:
int_type = IntegerType()
int_type


IntegerType

In [49]:
array_type = ArrayType(IntegerType())
array_type

ArrayType(IntegerType,true)

## Convertendo os tipos de Colunas

In [50]:
df.dtypes

[('nome', 'string'), ('id', 'string')]

In [51]:
df.select('nome', col('id').cast(IntegerType())).dtypes

[('nome', 'string'), ('id', 'int')]

### forma alternativa
IntegerType() == 'int'
StringType() == 'string'
FloatType() == 'float'

ArrayType(IntegralType()) == 'ARRAY<INT>'

In [52]:
df.select('nome', col('id').cast('int')).dtypes


[('nome', 'string'), ('id', 'int')]

## Schema e Criação de DataFrames
Um schema no Spark é uma especificação de tipos das colunas de um DataFrame. Eles são usados na leitura de dados externos e criação de DataFrames, e podem ser passados diretamente ao Spark ou podem ser inferidos. Passar um schema na leitura traz benefícios interessantes, como:

Evita que o Spark faça inferência de tipos, o que é custoso e demorado dependendo do tamanho do arquivo, além de propenso a erros;
Permite que o usuário identifique erros nos dados logo na leitura, caso os dados não sigam o schema especificado.

In [53]:

df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = ['nome', 'id'])

In [54]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
|  Pedro|  1|
|   João|  6|
|Juliana|  4|
|  Lucas|  7|
+-------+---+



In [55]:
df.dtypes

[('nome', 'string'), ('id', 'bigint')]

## Criando Schemas programaticamente

In [56]:
schema = StructType([
    StructField('nome', StringType()),
    StructField('id', IntegerType())
])

In [57]:
df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = schema)

In [58]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## Criando schemas com DDL

In [59]:
schema = 'nome STRING, id INT'


In [60]:
df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = schema)

In [61]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## Criando DataFrames

In [62]:
data = [('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)]

In [63]:
schema = 'nome STRING, id INT'


In [64]:
df = spark.createDataFrame(data, schema=schema)

In [65]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

In [66]:
df.schema


StructType(List(StructField(nome,StringType,true),StructField(id,IntegerType,true)))

In [67]:
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- id: integer (nullable = true)



In [68]:

spark.range(100).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



In [69]:
#df = spark.createDataFrame(pandas_df)

## Leitura e Escrita de Dados

### DataFrameReader
spark.read.format(format).option(args).load(file/path)

### DataFrameWriter
df.write.format(format).option(args).save(file/path)
Lendo e Escrevendo CSV
Opções mais comuns:

<li>header</li>
<li>inferSchema</li>
<li>sep</li>
<li>encoding</li>

In [70]:
data_path = './'

In [71]:
file_path = 'df_cnae.csv'

In [72]:
df = spark.read.format('csv').load(file_path)

In [73]:
df.limit(15).show()


+--------------------+
|                 _c0|
+--------------------+
|"0111301";"Cultiv...|
|"0111302";"Cultiv...|
|"0111303";"Cultiv...|
|"0111399";"Cultiv...|
|"0112101";"Cultiv...|
|"0112102";"Cultiv...|
|"0112199";"Cultiv...|
|"0113000";"Cultiv...|
|"0114800";"Cultiv...|
|"0115600";"Cultiv...|
|"0116401";"Cultiv...|
|"0116402";"Cultiv...|
|"0116403";"Cultiv...|
|"0116499";"Cultiv...|
|"0119901";"Cultiv...|
+--------------------+



## Definindo o schema¶

In [74]:
schema = 'cod_cnae STRING, descricao_cnae STRING'


## Opção de Correção 1

In [75]:
df = spark.read.csv(file_path, sep=';', encoding='ISO-8859-1', schema=schema)
df.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodÃ...|
+--------+--------------------+



## Opção de Correção 2¶

In [76]:
df = (
    spark.read
    .format('csv')
    .option('sep', ';')
    .option('encoding', 'ISO-8859-1')
    .schema(schema)
    .load(file_path)
)
df.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodÃ...|
+--------+--------------------+



## Opção de Correção 3

In [77]:
df = (
    spark.read
    .format('csv')
    .options(sep=';', encoding='ISO-8859-1')
    .schema(schema)
    .load(file_path)
)
df.limit(5).show()


+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodÃ...|
+--------+--------------------+



Obs: utilizando o método "options" podemos parametrizar melhor nossa função usando um dicionário


In [78]:
options_dict = {
    'sep': ';',
    'encoding': 'ISO-8859-1',
}

df = (
    spark.read
    .format('csv')
    .options(**options_dict)
    .schema('cod_cnae INT, descricao_cnae STRING')
    .load(file_path)
)
df.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
|  111301|    Cultivo de arroz|
|  111302|    Cultivo de milho|
|  111303|    Cultivo de trigo|
|  111399|Cultivo de outros...|
|  112101|Cultivo de algodÃ...|
+--------+--------------------+



In [79]:

df.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao_cnae: string (nullable = true)



In [80]:
#df.write.format('csv').save('./' + 'df_cnaet_este', header=True)

In [81]:
#spark.read.format('csv').load(data_path + 'df_cnaet_este', header=True).printSchema()


## Lendo e Escrevendo JSON

In [82]:
df.write.format('json').save(data_path + 'df_cnae.json')

AnalysisException: path file:/E:/bootcamp-dados-igti/semana02/df_cnae.json already exists.

In [ ]:

df_json = spark.read.format('json').load(data_path + 'df_cnae.json')

In [ ]:
df_json.show()

In [ ]:
df_json.printSchema()

## Lendo e Escrevendo ORC

In [ ]:
formato = 'orc'
df.write.format(formato).save(data_path + 'df_cnae.' + formato)

In [ ]:
df_orc = spark.read.format(formato).load(data_path + 'df_cnae.orc')

In [ ]:
df_orc.printSchema()

## Lendo e Escrevendo Parquet
Armazenamento colunar, em contraste com o CSV, que armazena baseado nas linhas. Assim, quando uma query é realizada é possível ignorar os dados não relevantes de maneira rápida e fácil, resultando em operações bem mais eficientes;
Preservação de metadados, incluindo os tipos das colunas, o que garante eficiência e praticidade na escrita e leitura (não é necessário especificar schemas para arquivos parquet);
Suporte a dados estruturados de forma aninhada, como listas;
Otimizado para processar dados particionados com volume na casa dos gigabytes para cada arquivo;
Compressão de dados na escrita, de forma a ocupar menos espaço;
Integração com ferramentas como AWS Athena, Amazon Redshift Spectrum, Google BigQuery e Google Dataproc.


In [83]:
df.write.format('parquet').save(data_path + 'df_cnae')

AnalysisException: path file:/E:/bootcamp-dados-igti/semana02/df_cnae already exists.

In [84]:
df_parquet = spark.read.format('parquet').load(data_path + 'df_cnae')

In [ ]:
df_parquet.printSchema()


mode:

append: arquivos empilhados aos ja existentes
ignore: retorna um erro silencioso
overwrite: sobrescreve os dados já existente
error (default): retorne erro se já existem dados

In [ ]:
df.write.format('parquet').mode('overwrite').save(data_path + 'df_cnae')